In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from crewai import LLM

llm = LLM(
    model="gemini/gemini-2.5-flash",
    temperature=0.1
)

In [7]:
from crewai.tools import BaseTool


class ReplaceJargonsTool(BaseTool):
    name: str = "Jargon replacement tool"
    description: str = "Replaces jargon with more specific terms. "

    def _run(self, email: str) -> str:
        replacements = {
            "PRX": "Project Phoenix (internal AI revamp project)",
            "TAS": "technical architecture stack",
            "DBX": "client database cluster",
            "SDS": "Smart Data Syncer",
            "SYNCBOT": "internal standup assistant bot",
            "WIP": "in progress",
            "POC": "proof of concept",
            "ping": "reach out"
        }

        suggestions = []
        email_lower = email.lower()
        for jargon, replacement in replacements.items():
            if jargon.lower() in email_lower:
                suggestions.append(f"Consider replacing '{jargon}' with '{replacement}'")
        return "\n".join(suggestions) if suggestions else "No jargon or internal abbreviations detected."


jt = ReplaceJargonsTool()

original_email = """
looping in Priya. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.
Let's sync up tomorrow if SYNCBOT allows 😄. ping me if any blockers.
"""

jt.run(original_email)

Using Tool: Jargon replacement tool


"Consider replacing 'PRX' with 'Project Phoenix (internal AI revamp project)'\nConsider replacing 'TAS' with 'technical architecture stack'\nConsider replacing 'SDS' with 'Smart Data Syncer'\nConsider replacing 'SYNCBOT' with 'internal standup assistant bot'\nConsider replacing 'ping' with 'reach out'"

In [9]:
from crewai import Agent, Task, Crew

email_assistant = Agent(
    role="Email Assistant Agent",
    goal="Improve emails and make them sound professional and clear",
    backstory="A highly experienced communication expert skilled in professional email writing",
    verbose=True,
    tools=[jt],
    llm=llm
)

email_task = Task(
    description=f"""Take the following rough email and rewrite it into a professional and polished version.
    Expand abbreviations:
    '''{original_email}'''""",
    agent=email_assistant,
    expected_output="A professional written email with proper formatting and content.",
)

crew = Crew(
    agents=[email_assistant],
    tasks=[email_task],
    verbose=True,
)

result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 17dad533-e14c-4cfe-989c-572e3d68a056                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Task: Take the following rough email and rewrite it into a professional and polished version.                  │
│      Expand abbreviations:                                                                                      │
│      '''                                                                                                        │
│  looping in Priya. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.                      │
│  Let's sync up tomorrow if SYNCBOT allows 😄. ping me if any blockers.                                          │
│  '''                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Thought: Thought: The user wants me to rewrite a rough email into a professional and polished version,         │
│  expanding abbreviations and improving clarity. I have identified several informal phrases, abbreviations, and  │
│  jargon. I will use the `Jargon replacement tool` to address the jargon first, then I will manually expand      │
│  abbreviations and refine the overall tone and structure.                                                       │
│  Let's apply the `Jargon replacement tool` to the entire email.                                                 │
│                                                                                                                 │
│  Using Tool: Jargon replacement tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: dfc8238f-34b7-4022-a04e-67fbb2bf0c7c                                                                     │
│  Agent: Email Assistant Agent                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

╭────────────────────────── Tracing Preference Saved ──────────────────────────╮
│                                                                              │
│  Info: Tracing has been disabled.                                            │
│                                                                              │
│  Your preference has been saved. Future Crew/Flow executions will not        │
│  collect traces.                                                             │
│                                                                              │
│  To enable tracing later, do any one of these:                               │
│  • Set tracing=True in your Crew/Flow code                                   │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file               │
│  • Run: crewai traces enable                                                 │
│                                       

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 17dad533-e14c-4cfe-989c-572e3d68a056                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Updates on Technical Architecture Stack, Project Phoenix, and Smart Data Syncer         │
│  Integration                                                                                                    │
│                                                                                                                 │
│  Dear Team,                                                                                                     │
│                                                                                                                 │
│  I am including Priya in this communication for her awareness.                                                  │
│                                                                                                                 │
│  Updates regarding the technical architecture stack and Project Phoenix (internal AI revamp project) are        │
│  available in the presentation deck.                                                                            │
│                                                                                                                 │
│  The Estimated Time of Arrival for the Smart Data Syncer integration is scheduled for Friday.                   │
│                                                                                                                 │
│  Let's plan to synchronize tomorrow to discuss these items further, pending availability of our internal        │
│  standup assistant bot. Please reach out to me if you encounter any obstacles.                                  │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯